In [3]:
import tensorflow as tf
import numpy as np
import os
import pickle
from sklearn.model_selection import train_test_split
from tensorflow import keras
from tensorflow.keras import datasets, layers, models

# unpack the pickle
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
dataset_dir = '/content/drive/MyDrive/FALL_2022_SEMESTER/COMS_4995_Applied_Deep_Learning/Course_Project/music_dataset/'
pickle_file_path = os.path.join(dataset_dir, 'mel_chroma_pic_waug.pickle')

In [5]:
# unpack the pickle file
pickle_in = open(pickle_file_path, "rb")
test_data = pickle.load(pickle_in)

In [6]:
# map each raga str to an int
raga_to_int = {
    'Sankarabaranam': 0,
    'Reethigowla': 1,
    'Kalyani': 2,
    'Shanmukhapriya': 3,
    'SindhuBhairavi': 4,
    'Sri': 5,
    'Surutti': 6,
    'Thodi': 7,
    'PoorviKalyani': 8,
    'Mohanam': 9,
    'Madhyamavathi': 10,
    'Kharaharapriya': 11,
    'Keeravani': 12,
    'Kapi': 13,
    'Kambhodhi': 14,
    'Kamas': 15, 
    'Jaganmohini': 16,
    'Hamsadhwani': 17,
    'Bilahari': 18,
    'Bhairavi': 19
}


In [7]:
# EXTRA step - try reversing the arrays? See how well that does?
# try upping the pitch
# try modulating volume
# 

In [8]:
# OK so now just create a random network and see how it does
# split up into test and train data
X = []
y = []
for val in test_data:
  X.append(val[0])
  y.append(raga_to_int[val[1]])

del test_data




In [9]:
X_train, X_test, y_train, y_test = train_test_split(X, y,
                                   random_state=104, 
                                   test_size=0.25, 
                                   shuffle=True)
X_train =np.array(X_train)
X_test =np.array(X_test)
y_train =np.array(y_train)
y_test =np.array(y_test)

In [10]:
print(f'X_train shape: {X_train.shape}')
print(f'X_test shape: {X_test.shape}')
print(f'y_train shape: {y_train.shape}')
print(f'y_test shape: {y_test.shape}')

X_train shape: (1199, 12, 2584)
X_test shape: (400, 12, 2584)
y_train shape: (1199,)
y_test shape: (400,)


In [11]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(12, 2584)),
  tf.keras.layers.Dense(256, activation='relu'),
  tf.keras.layers.Dropout(0.4),
  tf.keras.layers.Dense(20)
])

# for Inception, reshape as 76x408
"""
X_train = X_train.reshape(782, 76, 408)
X_test = X_test.reshape(782, 76, 408)

model = tf.keras.applications.InceptionV3(
    #include_top=True,
    weights=None,
    input_tensor=None,
    input_shape=(782, 76, 408),
    pooling=None,
    classes=1000,
    classifier_activation="softmax",
)
"""
"""model = models.Sequential()
model.add(layers.Conv2D(16, (3, 3), activation='relu', input_shape=(1, 12, 2584)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(8, (3, 3), activation='relu'))"""
#model.add(layers.MaxPooling2D((2, 2)))
#model.add(layers.Conv2D(32, (3, 3), activation='relu'))

"model = models.Sequential()\nmodel.add(layers.Conv2D(16, (3, 3), activation='relu', input_shape=(1, 12, 2584)))\nmodel.add(layers.MaxPooling2D((2, 2)))\nmodel.add(layers.Conv2D(8, (3, 3), activation='relu'))"

In [12]:
predictions = model(X_train[:1]).numpy()
predictions

array([[-0.33879432,  0.20026779,  0.34684503, -0.60153997,  0.06543832,
        -0.55451626, -0.4643549 ,  1.0392909 ,  0.62133956,  0.6582306 ,
        -0.03221151,  0.54870087, -0.57114   , -0.1915387 ,  0.01838827,
         0.6282417 , -0.5597275 ,  0.11164537,  1.0429568 ,  0.0385551 ]],
      dtype=float32)

In [13]:
tf.nn.softmax(predictions).numpy()


array([[0.02812649, 0.04822001, 0.05583224, 0.02162752, 0.04213778,
        0.02266881, 0.02480763, 0.11158619, 0.0734677 , 0.07622862,
        0.03821755, 0.06832031, 0.02229509, 0.03258877, 0.04020111,
        0.07397655, 0.02255099, 0.04413052, 0.111996  , 0.04102007]],
      dtype=float32)

In [14]:
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)


In [15]:

model.compile(optimizer='adam',
              loss=loss_fn,
              metrics=['accuracy'])

In [16]:
model.fit(X_train, y_train, epochs=30)


Epoch 1/30
38/38 [==============================] - 3s 51ms/step - loss: 8.1174 - accuracy: 0.1209
Epoch 2/30
38/38 [==============================] - 3s 72ms/step - loss: 2.7835 - accuracy: 0.2252
Epoch 3/30
38/38 [==============================] - 3s 90ms/step - loss: 2.5470 - accuracy: 0.2752
Epoch 4/30
38/38 [==============================] - 2s 52ms/step - loss: 2.4682 - accuracy: 0.2594
Epoch 5/30
38/38 [==============================] - 3s 78ms/step - loss: 2.2774 - accuracy: 0.2794
Epoch 6/30
38/38 [==============================] - 4s 103ms/step - loss: 2.2587 - accuracy: 0.2736
Epoch 7/30
38/38 [==============================] - 5s 122ms/step - loss: 2.1685 - accuracy: 0.3194
Epoch 8/30
38/38 [==============================] - 2s 63ms/step - loss: 2.0651 - accuracy: 0.3394
Epoch 9/30
38/38 [==============================] - 2s 51ms/step - loss: 1.9720 - accuracy: 0.3661
Epoch 10/30
38/38 [==============================] - 2s 50ms/step - loss: 1.9556 - accuracy: 0.3636
Epoch 1

In [17]:
# might have started to overfit on trarin accuracy after 1 epoch???
# try testing it 
model.evaluate(X_test,  y_test, verbose=2)


13/13 - 0s - loss: 2.3168 - accuracy: 0.3650 - 326ms/epoch - 25ms/step


[2.3167645931243896, 0.36500000953674316]

In [18]:
# ok so now, get the actual features of the dataset (Spectral centroid, spectral bandwidth, mel-spectrogram, Mel freq cepstral coeff, chroma)
# Get those into a pickle file, and see how it performs on the same neural network. If accuracy is low, then either modify network or add more data samples